## Tutorial: Quantum Adder
An adder has two basic building blocks:
1. Addition modulo 2
2. Carry output

For a quantum computer, both blocks can be implemented using CNOT gate.

In [1]:
from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.ops import X, CNOT, Measure, All, ControlledGate

def qsum_(c, a, b):
    CNOT|(a, b)    # CNOT (control bit, target bit)
    CNOT|(c, b)
    return b       # b -> b' = (a+b+c mod 2)

The quantum circuit of qsum_ is:
<img src="Images/sum.png" width="30%">

In [5]:
def qcarry_(eng, a, b, c):
    ancilla = eng.allocate_qubit()
    ControlledGate(X,2) | (a,b,ancilla) # a,b are control qubits, ancilla is a target qubit
    CNOT|(a,b)
    ControlledGate(X,2) | (c,b,ancilla)
    return (c,a,b,ancilla)  # (carry, a, a+b mod 2, next carry)
                            # (a+b mod 2) can be uncomputed later using CNOT

The quantum circuit of qcarry_ is:
<img src="Images/carry.png" width="27%">

Finally we can start to construct a quantum adder:
1. The input are two n-qubit quantum states, the output is a (n+1)-qubit quantum state which is converted into an integer.
2. We start from least significant bits $a_0$ and $b_0$, compute the carry $c_0$ and pass it to next bit.
3. The most significant bit is the carry $c_n$.
4. We uncompute registers-b, then use qsum_ to get less significant bits.

In [3]:
def qadd_(eng, qureg_A, qureg_B):
    assert len(qureg_A)==len(qureg_B)
    n=len(qureg_A)
    carries = [None for i in range(n+1)]
    carries[0] = eng.allocate_qubit()
    for i in range(n):
        _, _, _, carries[i+1] = qcarry_(eng,qureg_A[i],qureg_B[i],carries[i])
        CNOT|(qureg_A[i],qureg_B[i])      # uncompute a+b mod 2 to b
        qsum_(carries[i],qureg_A[i],qureg_B[i])
        
    All(Measure)|qureg_B # less significant bits
    Measure|carries[n]   # most significant bit
    eng.flush()
    
    result = ''
    for bit in qureg_B:
        result += str(int(bit))
    result += str(int(carries[n]))
    return result[::-1] # reverse

A sample test run

In [4]:
if __name__ == "__main__":
    eng = MainEngine()
    qureg_A=eng.allocate_qureg(3)
    X|qureg_A[0]
    X|qureg_A[1]
    X|qureg_A[2]
    qureg_B=eng.allocate_qureg(3)
    X|qureg_B[2]
    print(qadd_(eng,qureg_A,qureg_B))

    del eng

(Note: This is the (slow) Python simulator.)
1011
